In [1]:
%%HTML
<iframe width="400" height="225" src="https://www.youtube.com/embed/IVrGz8w0H8c?list=PLeo1K3hjS3utzQYDNRNluzqJqpMXx6hHu"
allowfullscreen></iframe>

# Repeated test code
`src/mydb.py` is a fake database that returns several simple values for fake employees. It mimics sql behavior. The simple test case syntax would read:

In [ ]:
def test_johns_id():
    db = MyDB()
    conn = db.connect('server')
    cur = conn.cursor()
    id = cur.execute("select id from employee_db where name=John")
    assert id == 123
def test_johns_id():
    db = MyDB()
    conn = db.connect('server')
    cur = conn.cursor()
    id = cur.execute("select id from employee_db where name=Tom")
    assert id == 789

However, the first three lines of each test case are repeated. Additionally, an sql connection is an expensive resource

# Setup and teardown via global variables
By defining the `setup_module` and `teardown_module` functions, the code can be reduced so that steps are not repeated. Note the use of global variables:

In [ ]:
conn = None  # Global
cur = None  # Global

def setup_module(module):
    global conn
    global cur
    db = MyDB()
    conn = db.connect('server')
    cur = conn.cursor()

def teardown_module(module):
    cur.close()
    conn.close()

def test_johns_id():
    id = cur.execute("select id from employee_db where name=John")
    assert id == 123

def test_johns_id():
    id = cur.execute("select id from employee_db where name=Tom")
    assert id == 789

# Defining a fixture
the `pytest.fixture` decorator can be used to return the same cursor as before for every test. Note the use of `cur` being passed to each test, and that `cur` is being passed to each test, not the return value `curs`

In [ ]:
@pytest.fixture
def cur():
    print("setting up")
    db = MyDB()
    conn = db.connect("server")
    curs = conn.cursor()
    return curs

def test_johns_id(cur):
    id = cur.execute("select id from employee_db where name=John")
    assert id == 123

def test_toms_id(cur):
    id = cur.execute("select id from employee_db where name=Tom")
    assert id == 789

However, in this case, a new cursor is being created for each test. By running `pytest -v --capture=no`, which allows the printing to go to the anaconda terminal, the following results are obtained:

Note that the setting up command was called twice, such that this fixture gets created anew for each test, which is not desired for the database cursor

# Module scope in a fixture
My using the `scope='module'` keyword argument, the setup can be configured to only run at the beginning of the test module instead of before each test:

In [ ]:
@pytest.fixture(scope='module')
def cur():
    print("setting up")
    db = MyDB()
    conn = db.connect("server")
    curs = conn.cursor()
    return curs

This yields the setup only happening once:

# Teardown via yield operator
By using the `yield` operator in the fixture, the shutdown can be handled automatically. This is similar to a context manager:

In [ ]:
@pytest.fixture(scope='module')
def cur():
    print("setting up")
    db = MyDB()
    conn = db.connect("server")
    curs = conn.cursor()
    yield curs
    print("closing down")
    curs.close()
    conn.close()
    
def test_johns_id(cur):
    id = cur.execute("select id from employee_db where name=John")
    assert id == 123


def test_toms_id(cur):
    id = cur.execute("select id from employee_db where name=Tom")
    assert id == 789